# Compute the total.

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import os

# Configure pararmters

In [2]:
metaPath = "../../metadata/Flickr_FL_Predictions_2014_2019_CLIP-BRF-ZS_1kmGrids.csv"
predLabel = "CLIP-BRF-ZS"
years_to_filter = [2014, 2015, 2016, 2017, 2018, 2019]

In [3]:
#The rule for differernt shape layers.

config = {
    "StateServerData": [
        ["SITE_NAME.notna()"] #State Park #SITE_NAME #First
    ],
    "NationalServerData": [
        ["PARKNAME.notna()", #National Park #PARKNAME #Second
         "SITE_NAME.isna()"] #State Park #SITE_NAME
    ],
    "ParkServerData": {
        "rules": [
            ["NAME20.notna()",  #Urban greenspaces #Urban Area #NAME20
             "Park_Name.notna()", #Park Server #Park_Name  
             "PARKNAME.isna()", #National Park #PARKNAM
             "SITE_NAME.isna()"], #State Park #SITE_NAME
            ["NAME20.isna()",  #Non-urban greenspace, but protect area be priority. #Urban Area #NAME20
             "Park_Name.notna()", #Park Server #Park_Name
             "IUCN_Cat.isna()", #Protected Area #IUCN_Cat
             "PARKNAME.isna()", #National Park #PARKNAME
             "SITE_NAME.isna()"] #State Park #SITE_NAME
        ],
        "subsplits": {
            "Urban_ParkServerData": ["NAME20.notna()"], #Urban_green space
            "OutUrban_ParkServerData": ["NAME20.isna()"] #Outside of Urban_green space
        }
    },
    "ProtectedArealData": [
        ["NAME20.notna()",  #Protected area in urban, but greenspace (park) be priority). #Urban Area #NAME20
         "Park_Name.isna()", #Park Server #Park_Name
         "PARKNAME.isna()", #National Park #PARKNAME
         "SITE_NAME.isna()", #State Park #SITE_NAME
         "IUCN_Cat.notna()"],#Protected Area #IUCN_Cat
        ["NAME20.isna()", #Protecte area in non-urban. #Urban Area #NAME20
         "PARKNAME.isna()", #National Park #PARKNAME
         "SITE_NAME.isna()", #State Park #SITE_NAME
         "IUCN_Cat.notna()"] #Protected Area #IUCN_Cat
    ],
    "OtherAreaData": [
        ["NAME20.isna()", #Urban Area #NAME20
         "Park_Name.isna()", #Park Server #Park_Name
         "PARKNAME.isna()", #National Park #PARKNAME
         "SITE_NAME.isna()", #State Park #SITE_NAME
         "IUCN_Cat.isna()"] ,#Protected Area #IUCN_Cat
    ]
}

# The categories for pie (must match keys in your subsets dict).
categories = [
    "Urban_ParkServerData",
    "OutUrban_ParkServerData",
    "StateServerData",
    "NationalServerData",
    "ProtectedArealData",
    "OtherAreaData"
]

count_columns = [
    "Urban greenspaces",
    "Non-urban greenspaces",
    "State parks",
    "National parks",
    "Protected areas",
    "Other areas"
]

# List of class names
class_names = ["Total", "Other"]

class_names_map = {"Biking":"Total", "Boating":"Total", "Camping":"Total", "Fishing":"Total", "Hiking":"Total", 
                   "Horseback_Riding":"Total","Hunting":"Total", "Shelling":"Total", "Surfing":"Total", 
                   "Swimming":"Total", "Wildlife_Viewing":"Total","Landscape_Aesthetics":"Total", "Other":"Other"}

# NLCD class mapping
nlcd_class_map = {
    11: "Water", 12: "Water", 21: "Developed", 22: "Developed",
    23: "Developed", 24: "Developed", 31: "Barren", 41: "Forest",
    42: "Forest", 43: "Forest", 51: "Shrubland", 52: "Shrubland",
    71: "Herbaceous", 72: "Herbaceous", 73: "Herbaceous", 74: "Herbaceous",
    81: "Pasture/Hay", 82: "Cultivated Crops", 90: "Wetlands", 95: "Wetlands"
}

# Data preprocessing

In [4]:
metadata = pd.read_csv(metaPath)

# keep only selected years
metadata["takendate"] = pd.to_datetime(metadata["takendate"], errors="coerce")
metadata = metadata[metadata.takendate.dt.year.isin(years_to_filter)]

metadata[predLabel] = metadata[predLabel].map(class_names_map) #Rename to category level

# build GeoDataFrame with correct CRS (WGS84 degrees)
geo_df = gpd.GeoDataFrame(
    metadata,
    geometry=gpd.points_from_xy(metadata.longitude, metadata.latitude),
    crs="EPSG:4326"
)

In [5]:
metadata.columns, len(metadata)

(Index(['ownerid', 'latitude', 'longitude', 'takendate', 'SITE_NAME',
        'PARKNAME', 'Park_Name', 'IUCN_Cat', 'NAME20', 'NLCD_2023',
        'CLIP-BRF-ZS', 'index_right', 'id', 'left', 'top', 'right', 'bottom',
        'PUD_ID', 'ownerid_ID'],
       dtype='object'),
 590378)

In [6]:
geo_df[predLabel].value_counts()

CLIP-BRF-ZS
Total    310773
Other    279605
Name: count, dtype: int64

In [7]:
metadata_Total = geo_df[geo_df[predLabel] == 'Total'].drop_duplicates(subset=['PUD_ID'])
metadata_NoCES = geo_df[geo_df[predLabel] == 'Other'].drop_duplicates(subset=['PUD_ID'])

print(len(metadata_Total), len(metadata_NoCES))

66652 50381


In [9]:
def build_condition(df, cond_str):
    col, method = cond_str.split(".")
    method = method.replace("()", "")  # strip parentheses if present
    return getattr(df[col], method)()

def combine_conditions(df, rule_blocks):
    """Combine AND inside a block, OR across blocks"""
    or_blocks = []
    for and_block in rule_blocks:
        conds = [build_condition(df, c) for c in and_block]
        and_cond = conds[0]
        for c in conds[1:]:
            and_cond &= c
        or_blocks.append(and_cond)
    final_cond = or_blocks[0]
    for ob in or_blocks[1:]:
        final_cond |= ob
    return final_cond

def apply_config(df, config):
    results = {}
    for name, rule_def in config.items():
        # allow both old format (list) and dict format (with subsplits)
        if isinstance(rule_def, dict):
            main_rules = rule_def["rules"]
            subsplits = rule_def.get("subsplits", {})
        else:
            main_rules = rule_def
            subsplits = {}

        # main subset
        cond = combine_conditions(df, main_rules)
        subset = df[cond]
        results[name] = subset

        # subsplits (optional)
        for sub_name, sub_rule in subsplits.items():
            sub_cond = combine_conditions(subset, [sub_rule])  # treat single split as one block
            results[sub_name] = subset[sub_cond]
    return results

subsets = apply_config(geo_df, config)

for name, df in subsets.items():
    print(name, len(df))
print("Total:", sum(len(df) for df in subsets.values()))



StateServerData 31642
NationalServerData 46148
ParkServerData 164732
Urban_ParkServerData 155840
OutUrban_ParkServerData 8892
ProtectedArealData 163862
OtherAreaData 183994
Total: 755110


<h1>Urban realted percentage<h1/>

In [10]:
results_total, results_owners, results_avg, results_avgdes = [], [], [], []

for cla in class_names:
    row_total, row_total_pct = [], []
    row_owners, row_owners_pct = [], []
    row_avg, row_avg_pct = [], []
    row_avgdes, row_avgdes_pct = [], []

    # --- collect raw values first ---
    total_vals, owner_vals, avg_vals, avgdes_vals = [], [], [], []

    for cat in categories:
        df_cat = subsets[cat]

        # metrics
        total_puds = len(df_cat[df_cat[predLabel] == cla].drop_duplicates(subset=['PUD_ID']))
        total_owns = len(df_cat[df_cat[predLabel] == cla].drop_duplicates(subset=['ownerid_ID']))
        avg_puds = total_puds / len(years_to_filter) if len(years_to_filter) > 0 else 0
        # denom = total_owns * len(years_to_filter) if len(years_to_filter) > 0 else 0
        # avg_puds_des = total_puds / denom if denom > 0 else 0
        total_puds_id = df_cat[df_cat[predLabel] == cla].drop_duplicates(subset=['PUD_ID'])['id'].value_counts()
        total_owner_id = df_cat[df_cat[predLabel] == cla].drop_duplicates(subset=['ownerid_ID'])['id'].value_counts()
        avg_puds_des_id = (
            total_puds_id / total_owner_id / len(years_to_filter)
            if len(years_to_filter) > 0 else pd.Series(dtype=float)
        )
        avg_puds_des = avg_puds_des_id.mean()  if avg_puds_des_id.mean() > 0 else 0

        total_vals.append(total_puds)
        owner_vals.append(total_owns)
        avg_vals.append(avg_puds)
        avgdes_vals.append(avg_puds_des)

    # --- compute percentages row-wise ---
    def to_percent(vals):
        row_sum = sum(vals)
        return [(v / row_sum * 100) if row_sum > 0 else 0 for v in vals]

    total_pct = to_percent(total_vals)
    owner_pct = to_percent(owner_vals)
    avg_pct = to_percent(avg_vals)
    avgdes_pct = to_percent(avgdes_vals)

    # append rows with counts + percentages
    results_total.append([cla] + total_vals + total_pct)
    results_owners.append([cla] + owner_vals + owner_pct)
    results_avg.append([cla] + avg_vals + avg_pct)
    results_avgdes.append([cla] + avgdes_vals + avgdes_pct)

# --- Build DataFrames ---
def build_df(results, name):
    return pd.DataFrame(
        results,
        columns=["Class"] + count_columns + [c + " %" for c in count_columns]
    )

df_total = build_df(results_total, "Total_PUDs")
df_owners = build_df(results_owners, "Total_Owners")
df_avg = build_df(results_avg, "Avg_PUDs")
df_avgdes = build_df(results_avgdes, "Avg_PUDs_Des")

In [11]:
# --- Save to Excel ---
with pd.ExcelWriter("../../metadata/CES_Urban_Distribution_Total.xlsx", engine="openpyxl") as writer:
    df_total.to_excel(writer, sheet_name="Total_PUDs", index=False)
    df_owners.to_excel(writer, sheet_name="Total_Owners", index=False)
    df_avg.to_excel(writer, sheet_name="Avg_PUDs", index=False)
    df_avgdes.to_excel(writer, sheet_name="Avg_PUDs_Des", index=False)

<h1>NLCD Percentage<h1/>

In [12]:
def NLCD_summary(geo_df):
    # Initialize summary DataFrame
    summary_total_puds = pd.DataFrame()
    summary_total_owns = pd.DataFrame()
    summary_avg_puds = pd.DataFrame()
    summary_avg_puds_des = pd.DataFrame()

    # Loop through each CES class
    for cla in class_names:
        total_puds_table = geo_df[geo_df[predLabel] == cla].drop_duplicates(subset=['PUD_ID'])
        total_owns_table = geo_df[geo_df[predLabel] == cla].drop_duplicates(subset=['ownerid_ID'])

        mapped_total_puds = total_puds_table["NLCD_2023"].map(nlcd_class_map).value_counts().sort_index()
        mapped_total_owns = total_owns_table["NLCD_2023"].map(nlcd_class_map).value_counts().sort_index()
        mapped_avg_puds = mapped_total_puds / len(years_to_filter) if len(years_to_filter) > 0 else 0
        
        mapped_avg_puds_des = {}
        # Create new columns with renamed NLCD classes
        total_puds_table["NLCD_rename"] = total_puds_table["NLCD_2023"].map(nlcd_class_map)
        total_owns_table["NLCD_rename"] = total_owns_table["NLCD_2023"].map(nlcd_class_map)

        nlcd_order = [
            "Developed",
            "Wetlands",
            "Water",
            "Pasture/Hay",
            "Barren",
            "Forest",
            "Cultivated Crops",
            "Herbaceous",
            "Shrubland"
        ]

        for cat in nlcd_order:
            # Filter for current class
            puds_filtered = total_puds_table[total_puds_table["NLCD_rename"] == cat]
            owns_filtered = total_owns_table[total_owns_table["NLCD_rename"] == cat]

            # Count occurrences per id
            total_puds_id = puds_filtered["id"].value_counts()
            total_owner_id = owns_filtered["id"].value_counts()

            # Align IDs and compute ratio safely
            avg_puds_des_id = (
                (total_puds_id / total_owner_id).fillna(0) / len(years_to_filter)
                if len(years_to_filter) > 0 else pd.Series(dtype=float)
            )

            mapped_avg_puds_des[cat] = avg_puds_des_id.mean()      
        

        # Convert to DataFrame
        df_total_puds = mapped_total_puds.reset_index()
        df_total_puds.columns = ['class', f'{cla}_count']
        df_total_own = mapped_total_owns.reset_index()
        df_total_own.columns = ['class', f'{cla}_count']    
        df_avg_puds = mapped_avg_puds.reset_index()
        df_avg_puds.columns = ['class', f'{cla}_count']
        df_avg_puds_des = pd.Series(mapped_avg_puds_des).reset_index()
        df_avg_puds_des.columns = ['class', f'{cla}_count']    


        total_total_puds = df_total_puds[f'{cla}_count'].sum()
        df_total_puds[f'{cla}_percentage'] = (df_total_puds[f'{cla}_count'] / total_total_puds * 100).fillna(0).round(2)
        total_total_own = df_total_own[f'{cla}_count'].sum()
        df_total_own[f'{cla}_percentage'] = (df_total_own[f'{cla}_count'] / total_total_own * 100).fillna(0).round(2)   
        total_avg_puds = df_avg_puds[f'{cla}_count'].sum()
        df_avg_puds[f'{cla}_percentage'] = (df_avg_puds[f'{cla}_count'] / total_avg_puds * 100).fillna(0).round(2)
        total_avg_puds_des = df_avg_puds_des[f'{cla}_count'].sum()
        df_avg_puds_des[f'{cla}_percentage'] = (df_avg_puds_des[f'{cla}_count'] / total_avg_puds_des * 100).fillna(0).round(2)

        # Merge into the summary
        if summary_total_puds.empty:
            summary_total_puds = df_total_puds
        else:
            summary_total_puds = pd.merge(summary_total_puds, df_total_puds, on='class', how='outer', sort=False)

        if summary_total_owns.empty:
            summary_total_owns = df_total_own
        else:
            summary_total_owns = pd.merge(summary_total_owns, df_total_own, on='class', how='outer', sort=False)

        if summary_avg_puds.empty:
            summary_avg_puds = df_avg_puds
        else:
            summary_avg_puds = pd.merge(summary_avg_puds, df_avg_puds, on='class', how='outer', sort=False)

        if summary_avg_puds_des.empty:
            summary_avg_puds_des = df_avg_puds_des
        else:
            summary_avg_puds_des = pd.merge(summary_avg_puds_des, df_avg_puds_des, on='class', how='outer', sort=False)
    
    #print(mapped_avg_puds_des)
    # Fill missing values
    summary_total_puds = summary_total_puds.fillna(0)
    summary_total_owns = summary_total_owns.fillna(0)
    summary_avg_puds = summary_avg_puds.fillna(0)
    summary_avg_puds_des = summary_avg_puds_des.fillna(0)
    
    return summary_total_puds, summary_total_owns, summary_avg_puds, summary_avg_puds_des

def reshape_summary(summary_df, tag):
    """
    Reshape one summary DataFrame into wide format with counts + percentages.
    tag = "Total_PUDs", "Total_Owns", "Avg_PUDs", "Avg_PUDs_Des"
    """
    # Step 1: Set class index
    df = summary_df.set_index("class")
    
    # Step 2: Split count and percentage parts
    count_df = df.filter(like="_count").T
    percentage_df = df.filter(like="_percentage").T
    
    # Step 3: Clean row labels
    count_df.index = count_df.index.str.replace("_count", "")
    percentage_df.index = percentage_df.index.str.replace("_percentage", "")
    
    # Step 4: Add suffixes to columns so we don’t lose CES activity meaning
    count_df.columns = [f"{col}_count" for col in count_df.columns]
    percentage_df.columns = [f"{col}_percentage" for col in percentage_df.columns]
    
    # Step 5: Combine
    final_df = pd.concat([count_df, percentage_df], axis=1)
    
    # Step 6: Reorder by NLCD order
    nlcd_order = [
        "Developed",
        "Wetlands",
        "Water",
        "Pasture/Hay",
        "Barren",
        "Forest",
        "Cultivated Crops",
        "Herbaceous",
        "Shrubland"
    ]
    
    count_cols_sorted = [f"{cls}_count" for cls in nlcd_order if f"{cls}_count" in final_df.columns]
    percent_cols_sorted = [f"{cls}_percentage" for cls in nlcd_order if f"{cls}_percentage" in final_df.columns]
    final_df = final_df[count_cols_sorted + percent_cols_sorted]
    
    return final_df

summary_total_puds, summary_total_owns, summary_avg_puds, summary_avg_puds_des = NLCD_summary(geo_df)

# Apply to all four
final_total_puds = reshape_summary(summary_total_puds, "Total_PUDs")
final_total_owns = reshape_summary(summary_total_owns, "Total_Owns")
final_avg_puds = reshape_summary(summary_avg_puds, "Avg_PUDs")
final_avg_puds_des = reshape_summary(summary_avg_puds_des, "Avg_PUDs_Des")



In [13]:
# --- Save to Excel with four sheets ---
with pd.ExcelWriter("../../metadata/CES_NLCD_percentages_Total.xlsx", engine="openpyxl") as writer:
    final_total_puds.to_excel(writer, sheet_name="Total_PUDs")
    final_total_owns.to_excel(writer, sheet_name="Total_Owns")
    final_avg_puds.to_excel(writer, sheet_name="Avg_PUDs")
    final_avg_puds_des.to_excel(writer, sheet_name="Avg_PUDs_Des")


## Other Area NLCD

In [14]:
OtherAreaData = subsets['OtherAreaData']

In [15]:
other_total_puds, othe_total_owns, othe_avg_puds, othe_avg_puds_des = NLCD_summary(OtherAreaData)

# Apply to all four
other_final_total_puds = reshape_summary(other_total_puds, "Total_PUDs")
other_final_total_owns = reshape_summary(othe_total_owns, "Total_Owns")
other_final_avg_puds = reshape_summary(othe_avg_puds, "Avg_PUDs")
other_final_avg_puds_des = reshape_summary(othe_avg_puds_des, "Avg_PUDs_Des")

# --- Save to Excel with four sheets ---
with pd.ExcelWriter("../../metadata/CES_NLCD_Other_percentages_Total.xlsx", engine="openpyxl") as writer:
    other_final_total_puds.to_excel(writer, sheet_name="Total_PUDs")
    other_final_total_owns.to_excel(writer, sheet_name="Total_Owns")
    other_final_avg_puds.to_excel(writer, sheet_name="Avg_PUDs")
    other_final_avg_puds_des.to_excel(writer, sheet_name="Avg_PUDs_Des")